In [ ]:
import numpy as np
import torch
from torch import nn


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()


class DenoiseAutoEncoder(nn.Module):
    def __init__(self):
        super(DenoiseAutoEncoder, self).__init__()
        # Encoder
        self.Encoder = nn.Sequential(
            # param [input_c, output_c, kernel_size, stride, padding]
            nn.Conv2d(3, 64, 3, 1, 1),   # [, 64, 96, 96]
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3, 1, 1), # [, 64, 96, 96]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),             # [, 64, 48, 48]
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3, 1, 1),  # [, 64, 48, 48]
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, 3, 1, 1), # [, 128, 48, 48]
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, 3, 1, 1), # [, 128, 48, 48]
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 256, 3, 1, 1), # [, 256, 48, 48]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                 # [, 256, 24, 24]
            nn.BatchNorm2d(256)   
        )
        
        # decoder
        self.Decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 3 ,1, 1),   # [, 128, 24, 24]
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.ConvTranspose2d(128, 128, 3, 2, 1, 1),   # [, 128, 48, 48]
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.ConvTranspose2d(128, 64, 3, 1, 1),    # [, 64, 48, 48]
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.ConvTranspose2d(64, 32, 3, 1, 1),      # [, 32, 48, 48]
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.ConvTranspose2d(32, 32, 3, 1, 1),      # [, 32, 48, 48]
            nn.ConvTranspose2d(32, 16, 3, 2, 1, 1),  # [, 16, 96, 96]
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.ConvTranspose2d(16, 3, 3, 1, 1),         # [, 3, 96, 96]
            nn.Sigmoid()
        )
    
    def forward(self, x):
        encoder = self.Encoder(x)
        decoder = self.Decoder(encoder)
        return encoder, decoder
        
# 输出网络结构
DAEmodel = DenoiseAutoEncoder().to(device)
DAEmodel.load_state_dict(torch.load('autodecode.mdl'))
DAEmodel.eval()


: 

In [184]:
# 直接用coremltools，pytorch_1.12.1 可用
import torch
import coremltools as ct
import coremltools
random_input = torch.rand(1, 3, 96, 96) 
traced_model = torch.jit.trace(DAEmodel, random_input) 
model = ct.convert(
    model=traced_model,
    source="pytorch",
    inputs=[ct.ImageType(name="input_image", shape=random_input.shape, scale=1 / 255.0, color_layout=ct.colorlayout.RGB)],
    # inputs=[ct.TensorType(name="input_tensor",dtype=np.float32, shape=(1,3,96,96))],
    # outputs=[ct.TensorType(name="mid_res"), ct.ImageType(name="output_image",color_layout=ct.colorlayout.RGB)],
    outputs=[ct.TensorType(name="mid_res"), ct.TensorType(name="output_tensor",dtype=np.float32)],
    # outputs=["mid_res","output_tensor"],

    minimum_deployment_target=coremltools.target.iOS13,
    convert_to="neuralnetwork",

) 


# Set feature descriptions (these show up as comments in XCode)
# model.input_description["input_tensor"] = "a (1,3,96,96) shaped tensor transfered from an image"
# model.output_description["output_tensor"] = "a (1,3,96,96) tensor, *255 and transpose((1,2,0) to convert to an image"
model.author = "Alyosha"
model.license = "leigithub1024/xxx"
model.short_description = "lowlight image enhancement"
model.version = "1.0"
modelName = "imgDenoise1.mlmodel"
model.save(modelName)


# 将输出结果*255
spec = coremltools.utils.load_spec(modelName)
builder = coremltools.models.neural_network.NeuralNetworkBuilder(spec=spec)
builder.add_elementwise(name=f"xx", input_names=[f"mid_res"], output_name=f"mid_res_xx", mode="MULTIPLY", alpha=255)
builder.add_elementwise(name=f"multiply_xy_by_two_output_image", input_names=[f"output_tensor"], output_name=f"output_image", mode="MULTIPLY", alpha=255)
builder.set_output(output_names=['mid_res_xx',"output_image"], output_dims=[(1,256,24,24),(3,96,96)])
# builder.set_output(output_names=["output_image"], output_dims=[(3,96,96)])

model_spec = builder.spec
coremltools.models.utils.save_spec(model_spec, modelName)

print(model_spec.description.output)





Tuple detected at graph output. This will be flattened in the converted model.
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 181/181 [00:01<00:00, 179.34 ops/s]


[name: "mid_res_xx"
type {
  multiArrayType {
    shape: 1
    shape: 256
    shape: 24
    shape: 24
    dataType: DOUBLE
  }
}
, name: "output_image"
type {
  multiArrayType {
    shape: 3
    shape: 96
    shape: 96
    dataType: DOUBLE
  }
}
]


In [185]:
# 更改模型输出为图像
import coremltools
import coremltools.proto.FeatureTypes_pb2 as ft 
spec = coremltools.utils.load_spec(modelName)
output = spec.description.output[1]
import coremltools.proto.FeatureTypes_pb2 as ft
output.type.imageType.colorSpace = ft.ImageFeatureType.RGB
output.type.imageType.height = 96
output.type.imageType.width = 96
output.name = 'output_image'
coremltools.utils.save_spec(spec, modelName)
print(spec.description.input,spec.description.output)



[name: "input_image"
type {
  imageType {
    width: 96
    height: 96
    colorSpace: RGB
  }
}
] [name: "mid_res_xx"
type {
  multiArrayType {
    shape: 1
    shape: 256
    shape: 24
    shape: 24
    dataType: DOUBLE
  }
}
, name: "output_image"
type {
  imageType {
    width: 96
    height: 96
    colorSpace: RGB
  }
}
]


In [187]:
# 在这个网站验证模型结构：https://netron.app/
# 验证模型能不能跑
# Use PIL to load and resize the image to expected size
from PIL import Image
import matplotlib.pyplot as plt
model = coremltools.models.MLModel(modelName)

#####输入格式：tensor
# example_image = Image.open("1.jpg").resize((96, 96))
# img = cv2.imread('./1.jpg')
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# img = cv2.resize(img, (96, 96),interpolation = cv2.INTER_LINEAR)
# img = img.astype(np.float32) / 255.0
# # mean = np.array([0.485, 0.456, 0.406])
# # val = np.array([0.229, 0.224, 0.225])
# # img = (img - mean) / val
# print("gemfield debug img shape1: ",img.shape)
# img= img.astype(np.float32)
# img = img.transpose((2,0,1))
# print("gemfield debug img shape2: ",img.shape)
# img = np.expand_dims(img,axis=0)
# print("gemfield debug img shape3: ",img.shape)
# out_dict = model.predict({"input_tensor": img})


##### 输入格式：image
img = Image.open('1.jpg')
img = img.resize((96,96))
# img = img.astype(np.float32) / 255.0
out_dict = model.predict({"input_image": img})



#####输出格式：tensor
# res = out_dict["output_tensor"] 
# arr = np.array((res[0]*255), dtype=np.uint8)
# print("arr shape:", arr.shape)
# arr = arr.transpose((1,2,0))
# print("transposed_arr shape",arr.shape, arr)
# image = Image.fromarray(arr)
# image.show()



##### 输出格式：图像
# print(out_dict)
# output_pil_image = out_dict["output_image"]
# image = output_pil_image.convert("RGB")
# image_array = np.array(image)
# image_array = image_array * 255
# print(image_array)
out_dict['output_image'].show()  #这里发现输出是全黑的，找到问题了，coreml的那个input格式转换没调好

In [ ]:
##尝试使用 pth => onnx => mlmodel
#转换为onnx
dummy_input = torch.rand(1, 3, 96, 96) #这里高宽可能是颠倒的
input_names = ["gemfield_in"]
output_names = ["gemfield_out"]
torch.onnx.export(DAEmodel,
                  dummy_input,
                  "syszux_scene.onnx",
                  verbose=True,
                  input_names=input_names,
                  output_names=output_names)

In [ ]:
#验证使用onnx推理

import cv2
import onnxruntime
import numpy as np
import sys
import torch

from PIL import Image
from torchvision import transforms

session = onnxruntime.InferenceSession("./syszux_scene.onnx")
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[1].name
input_shape = session.get_inputs()[0].shape
print("gemfield debug required input shape", input_name,output_name,input_shape)

img = cv2.imread('./10.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#INTER_NEAREST, INTER_LINEAR, INTER_AREA, INTER_CUBIC
img = cv2.resize(img, (96, 96),interpolation = cv2.INTER_LINEAR)

img = img.astype(np.float32) / 255.0

mean = np.array([0.485, 0.456, 0.406])
val = np.array([0.229, 0.224, 0.225])
# img = (img - mean) / val
# print(img)

print("gemfield debug img shape1: ",img.shape)
img= img.astype(np.float32)
img = img.transpose((2,0,1))
#img = img.transpose((2,1,0))
print("gemfield debug img shape2: ",img.shape)
img = np.expand_dims(img,axis=0)
print("gemfield debug img shape3: ",img.shape)

res = session.run([output_name], {input_name: img})
arr = np.array((res[0][0]*255), dtype=np.uint8)
print("arr shape:", arr.shape)

arr = arr.transpose((1,2,0))
print("transposed_arr shape",arr.shape, arr)
image = Image.fromarray(arr)
image.save('image.jpg')



In [ ]:
#转换为mlmodel
#报错：No module named 'coremltools.converters.nnssa'，换了好几个环境也无法解决
from onnx_coreml import convert
model = convert(model='syszux_scene.onnx',minimum_ios_deployment_target='13')
model.save('syszux_scene.mlmodel')


In [ ]:
#验证coreML推理